## Hill Climbing Algorithm for CartPole OpenAI Gym
This file implements the simplest Reinforcement Learning solution to the intro-to-RL CartPole problem from the OpenAI Gym (https://openai.com/requests-for-research/#cartpole), the "*hill-climbing algorithm*".

In [1]:
from __future__ import print_function
import gym
import numpy as np

In [2]:
env = gym.make('CartPole-v0')
print("Highs:", env.observation_space.high)
print("Lows: ", env.observation_space.low)

print(env.action_space)

[2016-07-12 00:59:45,964] Making new env: CartPole-v0


Highs: [  4.80000000e+00   3.40282347e+38   4.18879020e-01   3.40282347e+38]
Lows:  [ -4.80000000e+00  -3.40282347e+38  -4.18879020e-01  -3.40282347e+38]
Discrete(2)


In [3]:
observation = env.reset()
observation

array([ 0.03775622,  0.04316844,  0.00642069, -0.04089588])

In [4]:
# Here I'm just keeping track of my personal best. This has to be updated manually.
# ... I happened to randomly get a perfect model in the first 100 I tried (number 63)!
# They seem to happen in about 1/150 of the models I generate.
personal_best_reward = 1000
personal_best_weight = np.array([ 0.10047517,  0.45675998,  0.99510988,  0.75130867])

In [5]:
# HyperParameters:
learning_rate = 1  # There is no reason to keep this small, since we're perturbing by a random amount within this range.
                   # Since this isn't gradient descent and we only keep ones that perform better, you don't have the
                   # "overshooting" problem of too large a learning_rate. Here I've set it to the whole space.
num_runs = 10000
num_batches = 100
max_num_steps = 1000

In [6]:
def random_range(a,b,shape):
    return (b-a) * np.random.random(shape) + a

In [7]:
W_initial = random_range(-1,1,env.observation_space.shape)
W_initial

array([-0.18743879, -0.22296214,  0.24887228,  0.36269794])

In [8]:
def model_step(W,x):
    ''' Simplest model ever: Just the linear multiplication, i.e. dot product!
    Technically this is a logistic regression, which chooses between two classes. '''
    y = np.dot(W,x)
    return [0,1][y >= 0] # Use sign of result to decide left or right.
model_step(W_initial,observation)

0

In [9]:
def update_model(W_prev,score_prev, W, score):
    ''' Randomly perturb the weights, and if it performs better than last time, keep it. '''
    keep_W = W_prev
    keep_score = score_prev
    if score > score_prev:
        keep_W = W
        keep_score = score
        print("-- Replacing old model! {0} better than {1} --".format(score, score_prev))
    new_W = np.copy(keep_W)
    new_W += random_range(-learning_rate, learning_rate, new_W.shape)
    return new_W, keep_W, keep_score
Wb = random_range(-1,1,env.observation_space.shape)
new_W, prev_W, prev_score = update_model(W_initial,10, Wb, 12)
W_initial, Wb, new_W, prev_W, prev_score

-- Replacing old model! 12 better than 10 --


(array([-0.18743879, -0.22296214,  0.24887228,  0.36269794]),
 array([ 0.43936486,  0.94313114, -0.60929714,  0.14942928]),
 array([ 0.95948229,  1.38377709, -1.36906444,  0.89878144]),
 array([ 0.43936486,  0.94313114, -0.60929714,  0.14942928]),
 12)

In [10]:
def max_possible_reward():
    return max_num_steps

In [11]:
def test_model(W):
    total_reward = 0
    for i_episode in range(num_batches):
        observation = env.reset()
        done = False
        batch_reward = 0
        for _ in range(max_num_steps):
            #env.render()
            action = model_step(W,observation)
            observation, reward, done, info = env.step(action)
            batch_reward += reward
            if done:
                break
        #print("Batch Reward: {}".format(batch_reward))
        total_reward += batch_reward
    average_reward = total_reward/num_batches
    return total_reward,average_reward

test_model(W_initial)

(18595.0, 185.95)

In [12]:
prev_weights = W_initial
prev_reward = 0
W = np.copy(prev_weights)

In [13]:
for idx in range(num_runs):
    global prev_weights,prev_reward,W
    total_reward,average_reward = test_model(W)
    print("{0}/{1}: Average Reward: {2} Total Reward: {3}".format(idx, num_runs, average_reward, total_reward))
    W, prev_weights, prev_reward = update_model(prev_weights, prev_reward, W, total_reward)
    
    if average_reward == max_possible_reward():
        break
    
best_weights,best_weight_reward = W,total_reward
print("Best Reward:", best_weight_reward)
print("Best Weight:", best_weights)

if best_weight_reward > personal_best_reward:
    print("It's a NEW LAP RECORD!: {0}".format(best_weight_reward))
    print(best_weights)


0/10000: Average Reward: 218.75 Total Reward: 21875.0
-- Replacing old model! 21875.0 better than 0 --
1/10000: Average Reward: 32.57 Total Reward: 3257.0
2/10000: Average Reward: 9.94 Total Reward: 994.0
3/10000: Average Reward: 9.12 Total Reward: 912.0
4/10000: Average Reward: 475.67 Total Reward: 47567.0
-- Replacing old model! 47567.0 better than 21875.0 --
5/10000: Average Reward: 64.47 Total Reward: 6447.0
6/10000: Average Reward: 279.74 Total Reward: 27974.0
7/10000: Average Reward: 9.84 Total Reward: 984.0
8/10000: Average Reward: 32.27 Total Reward: 3227.0
9/10000: Average Reward: 20.13 Total Reward: 2013.0
10/10000: Average Reward: 338.02 Total Reward: 33802.0
11/10000: Average Reward: 55.66 Total Reward: 5566.0
12/10000: Average Reward: 42.19 Total Reward: 4219.0
13/10000: Average Reward: 58.15 Total Reward: 5815.0
14/10000: Average Reward: 323.63 Total Reward: 32363.0
15/10000: Average Reward: 463.32 Total Reward: 46332.0
16/10000: Average Reward: 38.31 Total Reward: 3831.0

In [14]:
best_weights

array([ 0.11671831,  0.1011577 ,  2.2157191 ,  0.29025011])

In [15]:
def render_model(W):
    total_reward = 0
    num_batches = 5
    for i_episode in range(num_batches):
        observation = env.reset()
        done = False
        batch_reward = 0
        print("{0}/{1}:".format(i_episode, num_batches))
        for _ in range(5000):
            #env.render()  # I don't think you can get this to render from MyBinder. :(
            action = model_step(W,observation)
            observation, reward, done, info = env.step(action)
            batch_reward += reward
            if done:
                break
        print("{0}/{1}: Batch Reward: {2}".format(i_episode, num_batches, batch_reward))
        total_reward += batch_reward
    average_reward = total_reward/num_batches
    return total_reward,average_reward

In [16]:
render_model(best_weights)

0/5:
0/5: Batch Reward: 5000.0
1/5:
1/5: Batch Reward: 5000.0
2/5:
2/5: Batch Reward: 5000.0
3/5:
3/5: Batch Reward: 5000.0
4/5:
4/5: Batch Reward: 5000.0


(25000.0, 5000.0)

In [17]:
render_model(personal_best_weight)

0/5:
0/5: Batch Reward: 5000.0
1/5:
1/5: Batch Reward: 5000.0
2/5:
2/5: Batch Reward: 5000.0
3/5:
3/5: Batch Reward: 5000.0
4/5:
4/5: Batch Reward: 5000.0


(25000.0, 5000.0)